# Use PMML to predict iris species with `ibm-watsonx-ai`

This notebook contains steps from storing sample PMML model to starting scoring new data using batch deployment. 

Some familiarity with python is helpful. This notebook uses Python 3.11.

You will use a **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict iris species.

## Learning goals

The learning goals of this notebook are:

-  Working with the WML instance
-  Batch deployment of PMML model
-  Scoring of deployed model


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Model upload](#upload) 
3. [Batch deployment creation](#deploy)
4. [Scoring](#score)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget| tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [1]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.0"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.0"
)

```

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in `watsonx.ai`, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.

**Action**: Download sample PMML model from git project using wget.

In [7]:
import os
from wget import download

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = download('https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd5.0/models/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store downloaded file in `watsonx.ai` repository.

In [8]:
sw_spec_uid = client.software_specifications.get_uid_by_name("pmml-3.0_4.3")

meta_props = {
    client.repository.ModelMetaNames.NAME: "pmmlmodel",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1'}

In [9]:
published_model = client.repository.store_model(model=filename, meta_props=meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 3. Create battch deployment

You can use command bellow to create batch deployment for stored model.

In [10]:
model_id = client.repository.get_model_id(published_model)
deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Sample PMML Batch deployment",
        client.deployments.ConfigurationMetaNames.BATCH:{},
        client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
            "name": "S",
            "num_nodes": 1
        }
    }
)



######################################################################################

Synchronous deployment creation for id: '03debeb3-befa-41cc-9b3f-baf3fedd51e3' started

######################################################################################


ready.


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='46bddf6a-f4c9-4e1f-a436-bee4cf2c6a03'
-----------------------------------------------------------------------------------------------




Batch deployment has been created.

You can retrieve now your deployment ID.

In [11]:
deployment_id = client.deployments.get_id(deployment)

You can also list all deployments in your space.

In [ ]:
client.deployments.list()

If you want to get additional information on your deployment, you can do it as below.

In [13]:
client.deployments.get_details(deployment_id)

{'entity': {'asset': {'id': '03debeb3-befa-41cc-9b3f-baf3fedd51e3'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'Sample PMML Batch deployment',
  'space_id': 'cbd87244-b000-4279-b991-3cefbf8b1555',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2024-04-24T13:41:59.345Z',
  'id': '46bddf6a-f4c9-4e1f-a436-bee4cf2c6a03',
  'modified_at': '2024-04-24T13:41:59.345Z',
  'name': 'Sample PMML Batch deployment',
  'owner': '1000330999',
  'space_id': 'cbd87244-b000-4279-b991-3cefbf8b1555'}}

<a id="score"></a>
## 4. Scoring

You can send new scoring records to batch deployment using by creating job.

In [14]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
            'values': [[5.1, 3.5, 1.4, 0.2]]
        }
    ]
}

job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)

Now, your job has been submitted to runtime.

You can retrieve now your job ID.

In [20]:
job_id = client.deployments.get_job_id(job)

You can also list all jobs in your space.

In [ ]:
client.deployments.list_jobs()

If you want to get additional information on your job, you can do it as below.

In [ ]:
client.deployments.get_job_details(job_id)

**Monitor job execution**  
Here you can check status of your batch scoring.

In [21]:
import time

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    job_details_do = client.deployments.get_job_details(job_id)
    print(job_details_do)
else:
    print("Job hasn't completed successfully in 5 minutes.")

 Current state: completed
{'entity': {'deployment': {'id': '46bddf6a-f4c9-4e1f-a436-bee4cf2c6a03'}, 'platform_job': {'job_id': '67ff6035-d444-4f29-8cc3-4a2e15109267', 'run_id': '57ef959f-1164-459f-8000-0e78e3a12cc1'}, 'scoring': {'input_data': [{'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}], 'predictions': [{'fields': ['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species'], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}], 'status': {'completed_at': '2024-04-24T13:57:08.000Z', 'running_at': '2024-04-24T13:57:08.000Z', 'state': 'completed'}}}, 'metadata': {'created_at': '2024-04-24T13:56:31.606Z', 'id': '67098cc3-b781-4557-9c9c-2194553885fa', 'modified_at': '2024-04-24T13:57:08.258Z', 'name': 'name_415333db-d56a-4745-b8bf-a47f2b0a2f45', 'space_id': 'cbd87244-b000-4279-b991-3cefbf8b1555'}}


**Get scored data**

In [22]:
import json

print(json.dumps(client.deployments.get_job_details(job_id), indent=2))

{
  "entity": {
    "deployment": {
      "id": "46bddf6a-f4c9-4e1f-a436-bee4cf2c6a03"
    },
    "platform_job": {
      "job_id": "67ff6035-d444-4f29-8cc3-4a2e15109267",
      "run_id": "57ef959f-1164-459f-8000-0e78e3a12cc1"
    },
    "scoring": {
      "input_data": [
        {
          "fields": [
            "Sepal.Length",
            "Sepal.Width",
            "Petal.Length",
            "Petal.Width"
          ],
          "values": [
            [
              5.1,
              3.5,
              1.4,
              0.2
            ]
          ]
        }
      ],
      "predictions": [
        {
          "fields": [
            "$R-Species",
            "$RC-Species",
            "$RP-Species",
            "$RP-setosa",
            "$RP-versicolor",
            "$RP-virginica",
            "$RI-Species"
          ],
          "values": [
            [
              "setosa",
              1.0,
              1.0,
              1.0,
              0.0,
              0.0,
   

As we can see this is Iris Setosa flower.

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

You successfully completed this notebook! You learned how to use `watsonx.ai` for PMML model deployment and scoring. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Jan Sołtysik**, Software Engineer at IBM.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.